In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import eda

from tqdm import tqdm

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id']

feat_no = '105_new_'

In [2]:
def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')

### Data Load

In [3]:
train = elo_load_data('train')
test = elo_load_data('test')
train_id = train[key].values
test_id = test[key].values
train_test = pd.concat([train, test], axis=0).reset_index(drop=True)
new = elo_load_data('new_')
# historical = elo_load_data('historical')

100%|██████████| 3/3 [00:00<00:00,  3.44it/s]


In [85]:
display(train.head())
print(train[key].drop_duplicates().shape)
print(train.shape)
display(new.head())
print(new.shape)
print(new[key].drop_duplicates().shape)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


(201917,)
(201917, 6)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1793298,Y,C_ID_00007093c1,76,N,1.0,B,222,M_ID_08f01305af,2,-0.671775,2018-04-03 11:13:35,3,2,21
1793299,Y,C_ID_00007093c1,69,N,1.0,B,879,M_ID_00a6ca8a8a,2,-0.656749,2018-04-09 16:23:59,1,9,29
38630,Y,C_ID_0001238066,314,N,1.0,B,307,M_ID_7d8102bb34,1,-0.732783,2018-03-01 16:48:27,1,9,19
38618,Y,C_ID_0001238066,314,N,1.0,B,367,M_ID_235e546dcc,1,-0.672136,2018-03-03 22:44:57,1,9,16
38633,Y,C_ID_0001238066,333,N,1.0,B,783,M_ID_a88790a464,1,-0.641722,2018-03-04 13:05:16,1,9,19


(1963031, 14)
(290001,)


### Target Join

In [5]:
# Target Join
new_target = new.merge(train[[key, target]], how='left', on=key)
try:
    if logger:
        pass
except NameError:
    logger=utils.logger_func()

2018-12-02 11:07:05,371 utils 366 [INFO]    [logger_func] start 


In [6]:
new_target.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,target
0,Y,C_ID_00007093c1,76,N,1.0,B,222,M_ID_08f01305af,2,-0.671775,2018-04-03 11:13:35,3,2,21,0.134077
1,Y,C_ID_00007093c1,69,N,1.0,B,879,M_ID_00a6ca8a8a,2,-0.656749,2018-04-09 16:23:59,1,9,29,0.134077
2,Y,C_ID_0001238066,314,N,1.0,B,307,M_ID_7d8102bb34,1,-0.732783,2018-03-01 16:48:27,1,9,19,NaN
3,Y,C_ID_0001238066,314,N,1.0,B,367,M_ID_235e546dcc,1,-0.672136,2018-03-03 22:44:57,1,9,16,NaN
4,Y,C_ID_0001238066,333,N,1.0,B,783,M_ID_a88790a464,1,-0.641722,2018-03-04 13:05:16,1,9,19,NaN


### Target EncodingのWeighted Mean

In [11]:
base = utils.read_df_pkl('../input/base0*').set_index(key)
df = new_target
weight = 0.95

# Weighted Meanする特徴リスト
feat_list = [col for col in df.columns if col.count('@')]
num_cols = ['installments', 'purchase_amount'] + feat_list

max_date = df.groupby(key)['purchase_date'].max().reset_index().rename(columns={'purchase_date':'latest_date'})
df_w = df.merge(max_date, how='inner', on=key)

df_w['diff_days'] = (df_w['latest_date'].map(lambda x: parse(x)) - df_w['purchase_date'].map(lambda x: parse(x))).map(lambda x: x.days)

for col in num_cols:
    col_name = f'W{weight}_{col}@'
    df_w['W'] = df_w['diff_days'].map(lambda x: np.power(weight, x))
    df_w[col_name] = df_w[col] * df_w['W']
    tmp = df_w.groupby(key)[col_name].sum() /  df_w.groupby(key)['W'].sum()
    tmp.name = col_name
    base = base.join(tmp, how='left')
    
train_id = train[key].values
test_id = test[key].values
for col in base.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj = base.loc[train_id, :][col].values, path=f'../features/1_first_valid/{feat_no}train_{col}')
        utils.to_pkl_gzip(obj = base.loc[test_id, :][col].values, path=f'../features/1_first_valid/{feat_no}test_{col}')

100%|██████████| 3/3 [00:00<00:00, 97.42it/s]


### Latest Feature
row_no==1のfeature

In [23]:
feat_no = '103_new_'

In [24]:
num_list = ['purchase_amount', 'installments']
tmp = new_target.query("row_no==1")
base = utils.read_df_pkl('../input/base0*').set_index(key)
base = base.join(tmp.set_index(key).drop(target, axis=1), how='left')
prefix = 'latest1_'

train_id = train[key].values
test_id = test[key].values
for col in base.columns:
    if col.count('@') or col in num_list:
        utils.to_pkl_gzip(obj = base.loc[train_id, :][col].values, path=f'../features/1_first_valid/{feat_no}train_{prefix}{col}')
        utils.to_pkl_gzip(obj = base.loc[test_id, :][col].values, path=f'../features/1_first_valid/{feat_no}test_{prefix}{col}')

100%|██████████| 3/3 [00:00<00:00, 86.77it/s]


row_no<=3のfeature

In [25]:
tmp = new_target.query("row_no<=3")
num_list = ['purchase_amount', 'installments']
feat_cols = [col for col in tmp.columns if col.count('@')] + num_list
agg_mean = tmp[[key]+feat_cols].groupby(key).mean()
agg_std = tmp[[key]+feat_cols].groupby(key).std()
agg_max = tmp[[key]+feat_cols].groupby(key).max()
agg_min = tmp[[key]+feat_cols].groupby(key).min()
agg_mean.columns = [f"{col}_mean@" for col in agg_mean.columns]
agg_std.columns = [f"{col}_std@" for col in agg_std.columns]
agg_max.columns = [f"{col}_max@" for col in agg_max.columns]
agg_min.columns = [f"{col}_min@" for col in agg_min.columns]

base = utils.read_df_pkl('../input/base0*').set_index(key)
for tmp in [agg_mean, agg_std, agg_max, agg_min]:
    base = base.join(tmp, how='left')
prefix = 'latest3_'

train_id = train[key].values
test_id = test[key].values
for col in base.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj = base.loc[train_id, :][col].values, path=f'../features/1_first_valid/{feat_no}train_{prefix}{col}')
        utils.to_pkl_gzip(obj = base.loc[test_id, :][col].values, path=f'../features/1_first_valid/{feat_no}test_{prefix}{col}')

100%|██████████| 3/3 [00:00<00:00, 99.48it/s]


### row_noを逆転させ、初回の特徴、初回と2回目のdiffを特徴にする

In [19]:
df_first = new_target.drop('row_no', axis=1)
df_first.sort_values(by=[key, 'purchase_date', 'merchant_id'], ascending=True, inplace=True)
df_first = utils.row_number(df=df_first, level=key)
df_first.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,diff_days_lag1@,diff_days_lag2@,diff_days_lag3@,ratio_days_lag1_2@,ratio_days_lag1_3@,ratio_days_lag2_3@,target,TE1208_city_id@,TE1208_category_1@,TE1208_category_2@,TE1208_category_3@,TE1208_merchant_category_id@,TE1208_state_id@,TE1208_subsector_id@,row_no
0,Y,C_ID_00007093c1,76,N,1.0,B,222,M_ID_08f01305af,2,-0.671775,2018-04-03 11:13:35,3,2,21,NaN,NaN,NaN,NaN,NaN,NaN,0.134077,-0.544011,-0.583303,-0.591474,-0.527694,-0.574022,-0.581858,-0.575213,1
1,Y,C_ID_00007093c1,69,N,1.0,B,879,M_ID_00a6ca8a8a,2,-0.656749,2018-04-09 16:23:59,1,9,29,6.0,NaN,NaN,NaN,NaN,NaN,0.134077,-0.544011,-0.583303,-0.591474,-0.527694,-0.502899,-0.581858,-0.505548,2
2,Y,C_ID_0001238066,314,N,1.0,B,307,M_ID_7d8102bb34,1,-0.732783,2018-03-01 16:48:27,1,9,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.609823,-0.583303,-0.591474,-0.527694,-0.575209,-0.581858,-0.575213,1
3,Y,C_ID_0001238066,314,N,1.0,B,367,M_ID_235e546dcc,1,-0.672136,2018-03-03 22:44:57,1,9,16,2.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.582444,-0.583303,-0.591474,-0.765577,-0.620861,-0.581858,-0.631858,2
4,Y,C_ID_0001238066,333,N,1.0,B,783,M_ID_a88790a464,1,-0.641722,2018-03-04 13:05:16,1,9,19,0.0,2.0,NaN,0.0,NaN,NaN,NaN,-0.791251,-0.929266,-0.761306,-0.527694,-0.748654,-0.761306,-0.631858,3


### First Transaction
row_no==1(reverse)

In [26]:
tmp = df_first.query("row_no==1")
base = utils.read_df_pkl('../input/base0*').set_index(key)
base = base.join(tmp.set_index(key).drop(target, axis=1), how='left')
prefix = 'first1_'

train_id = train[key].values
test_id = test[key].values
for col in base.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj = base.loc[train_id, :][col].values, path=f'../features/1_first_valid/{feat_no}train_{prefix}{col}')
        utils.to_pkl_gzip(obj = base.loc[test_id, :][col].values, path=f'../features/1_first_valid/{feat_no}test_{prefix}{col}')

100%|██████████| 3/3 [00:00<00:00, 83.63it/s]


#### 初回と2回目、3回目の比率、差分

In [27]:
num_list = [key, 'purchase_amount', 'installments']
row1 = df_first.query("row_no==1")[num_list].set_index(key).rename(columns={'purchase_amount':'amt_1', 'installments':'ins_1'})
row2 = df_first.query("row_no==2")[num_list].set_index(key).rename(columns={'purchase_amount':'amt_2', 'installments':'ins_2'})
row3 = df_first.query("row_no==3")[num_list].set_index(key).rename(columns={'purchase_amount':'amt_3', 'installments':'ins_3'})

row123 = row1.join(row2).join(row3)
row123['purchase_amount_1_div_2@'] = row123['amt_1'] / row123['amt_2']
row123['purchase_amount_1_div_3@'] = row123['amt_1'] / row123['amt_3']
row123['purchase_amount_1_diff_2@'] = row123['amt_1'] - row123['amt_2']
row123['purchase_amount_1_diff_3@'] = row123['amt_1'] - row123['amt_3']
row123['installments_1_div_2@'] = row123['ins_1'] / row123['ins_2']
row123['installments_1_div_3@'] = row123['ins_1'] / row123['ins_3']
row123['installments_1_diff_2@'] = row123['ins_1'] - row123['ins_2']
row123['installments_1_diff_3@'] = row123['ins_1'] - row123['ins_3']

base = utils.read_df_pkl('../input/base0*').set_index(key)
base = base.join(row123, how='left')
prefix = 'first3_'

train_id = train[key].values
test_id = test[key].values
for col in base.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj = base.loc[train_id, :][col].values, path=f'../features/1_first_valid/{feat_no}train_{prefix}{col}')
        utils.to_pkl_gzip(obj = base.loc[test_id, :][col].values, path=f'../features/1_first_valid/{feat_no}test_{prefix}{col}')

100%|██████████| 3/3 [00:00<00:00, 93.06it/s]


### Visualize

In [139]:
for i, cid in enumerate(train_id):
    tmp = new_target.set_index(key).loc[cid, :]
    try:
        plt.figure(figsize=(20, 5))
        sns.lmplot(data=tmp, x='purchase_date', y='purchase_amount')
        plt.show()
    except ValueError:
        pass
    if i==10:
        sys.exit()

TypeError: '<' not supported between instances of 'float' and 'str'